# Veri Arttırma

## Genel Bakış

Bu çalışma dosyası, veri arttırmayı gösterir: görüntü döndürme gibi rastgele (ancak gerçekçi) dönüşümler uygulayarak eğitim kümenizin çeşitliliğini artırmaya yönelik bir teknik.

Veri arttırmayı iki şekilde nasıl uygulayacağınızı öğreneceksiniz:

- `tf.keras.layers.Resizing`, `tf.keras.layers.Rescaling`, `tf.keras.layers.RandomFlip`, ve `tf.keras.layers.RandomRotation` gibi Keras ön işleme katmanlarını kullanın.

- `tf.image.flip_left_right`, `tf.image.rgb_to_grayscale`, `tf.image.adjust_brightness`, `tf.image.central_crop` ve `tf.image.stateless_random` gibi `tf.image` yöntemlerini kullanın.

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

## Bir veri kümesini indirin

Bu çalışma dosyası, [tf_flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers) veri kümesini kullanır. Kolaylık sağlamak için [TensorFlow Veri Kümelerini](https://www.tensorflow.org/datasets) kullanarak veri kümesini indirin. Verileri içe aktarmanın diğer yolları hakkında bilgi edinmek isterseniz, [load images](https://www.tensorflow.org/tutorials/load_data/images) eğitimine bakın.


In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Çiçekler veri kümesinde beş sınıf vardır.

In [ ]:
num_classes = metadata.features['label'].num_classes
print(num_classes)

Veri kümesinden bir görüntü alalım ve onu veri arttırmayı göstermek için kullanalım.

In [ ]:
get_label_name = metadata.features['label'].int2str

image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

## Keras ön işleme katmanlarını kullanın

### Yeniden boyutlandırma ve yeniden ölçeklendirme

Resimlerinizi tutarlı bir şekle (`tf.keras.layers.Resizing` ile) yeniden boyutlandırmak ve piksel değerlerini yeniden ölçeklendirmek (`tf.keras.layers.Rescaling` ile) için Keras ön işleme katmanlarını kullanabilirsiniz.

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

Not: Yukarıdaki yeniden ölçeklendirme katmanı, piksel değerlerini `[0, 1]` aralığında standart hale getirir. Bunun yerine `[-1, 1]` olmasını isteseydiniz, `tf.keras.layers.Rescaling(1./127.5, offset=-1)` yazardınız.

Bu katmanları bir görüntüye uygulamanın sonucunu görselleştirebilirsiniz.

In [ ]:
result = resize_and_rescale(image)
_ = plt.imshow(result)

Piksellerin `[0, 1]` aralığında olduğunu doğrulayın:

In [ ]:
print("Min and max pixel values:", result.numpy().min(), result.numpy().max())

### Veri arttırma

`tf.keras.layers.RandomFlip` ve `tf.keras.layers.RandomRotation` gibi veri arttırma için Keras ön işleme katmanlarını da kullanabilirsiniz.

Birkaç ön işleme katmanı oluşturalım ve bunları tekrar tekrar aynı görüntüye uygulayalım.

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [ ]:
# Add the image to a batch.
image = tf.cast(tf.expand_dims(image, 0), tf.float32)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

`tf.keras.layers.RandomContrast`, `tf.keras.layers.RandomCrop`, `tf.keras.layers.RandomZoom` ve diğerleri dahil olmak üzere veri arttırma için kullanabileceğiniz çeşitli ön işleme katmanları vardır.

### Keras ön işleme katmanlarını kullanmak için iki seçenek

Bu ön işleme katmanlarını önemli ödünleşimlerle birlikte kullanmanın iki yolu vardır.

#### Seçenek 1: Ön işleme katmanlarını modelinizin bir parçası yapın

In [ ]:
model = tf.keras.Sequential([
  # Add the preprocessing layers you created earlier.
  resize_and_rescale,
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # Rest of your model.
])

Bu durumda dikkat edilmesi gereken iki önemli nokta vardır:

* Veri arttırma, cihazda, diğer katmanlarınızla eşzamanlı olarak çalışır ve GPU hızlandırmasından yararlanır.

* Modelinizi `model.save` kullanarak dışa aktardığınızda, ön işleme katmanları modelinizin geri kalanıyla birlikte kaydedilir. Bu modeli daha sonra dağıtırsanız, görüntüleri otomatik olarak standartlaştıracaktır (katmanlarınızın yapılandırmasına göre). Bu, sizi bu mantığı sunucu tarafını yeniden uygulama zorunluluğundan kurtarabilir.

Not: Veri arttırma, test zamanında etkin değildir, bu nedenle girdi görüntüleri yalnızca `Model.fit` çağrıları sırasında artırılacaktır (`Model.evaluate` veya "`Model.predict` değil).

#### Seçenek 2: Ön işleme katmanlarını veri kümenize uygulayın

In [ ]:
aug_ds = train_ds.map(
  lambda x, y: (resize_and_rescale(x, training=True), y))

Bu yaklaşımla, yığın halinde artırılmış görüntü sağlayan bir veri kümesi oluşturmak için `Dataset.map` kullanırsınız. Bu durumda:

* Veri artırma, CPU'da eşzamansız olarak gerçekleşir ve engellemez. Aşağıda gösterilen `Dataset.prefetch`i kullanarak veri ön işleme ile modelinizin GPU'daki eğitimini çakıştırabilirsiniz.

* Bu durumda, `Model.save` çağırdığınızda önişleme katmanları modelle birlikte dışa aktarılmayacaktır. Kaydetmeden veya sunucu tarafında yeniden uygulamadan önce bunları modelinize eklemeniz gerekir. Eğitimden sonra, dışa aktarmadan önce ön işleme katmanlarını ekleyebilirsiniz.

### Ön işleme katmanlarını veri kümelerine uygulayın

Daha önce oluşturduğunuz Keras ön işleme katmanlarıyla eğitim, doğrulama ve test veri kümelerini yapılandırın. Ayrıca, I/O bloke olmadan diskten toplu işler elde etmek için paralel okumalar ve arabelleğe alınmış önceden getirme kullanarak veri kümelerini performans için yapılandıracaksınız. 

Not: Veri arttırma yalnızca eğitim kümesine uygulanmalıdır.

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

### Model Eğitimi

Tamamlanması için, şimdi az önce hazırladığınız veri kümelerini kullanarak bir modeli eğiteceksiniz.

[Sequential](https://www.tensorflow.org/guide/keras/sequential_model) model, her birinde maksimum ortaklama katmanı (`tf.keras.layers.MaxPooling2D`) bulunan üç evrişim bloğundan (`tf.keras.layers.Conv2D`) oluşur. Üstünde bir ReLU etkinleştirme işlevi (`'relu'`) tarafından etkinleştirilen 128 birimli tam bağlı bir katman (`tf.keras.layers.Dense`) vardır. Bu model doğruluk için ayarlanmamıştır (amaç size mekaniği göstermektir).

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

`tf.keras.optimizers.Adam` optimize edicisini ve `tf.keras.losses.SparseCategoricalCrossentropy` kayıp işlevini seçin. Her eğitim dönemi için eğitim ve doğrulama doğruluğunu görüntülemek için `metrics` bağımsız değişkenini `Model.compile` öğesine iletin.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Birkaç epoch eğitim başlatılması:

In [ ]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
loss, acc = model.evaluate(test_ds)
print("Accuracy", acc)

### Özel veri arttırma

Ayrıca özel veri arttırma katmanları da oluşturabilirsiniz.

Eğitimin bu bölümü, bunu yapmanın iki yolunu gösterir:

- İlk önce bir `tf.keras.layers.Lambda` katmanı oluşturacaksınız. Bu, kısa kod yazmanın iyi bir yoludur.
- Ardından, size daha fazla kontrol sağlayan [alt sınıflama (subclassing)](https://www.tensorflow.org/guide/keras/custom_layers_and_models) aracılığıyla yeni bir katman yazacaksınız.

Her iki katman da, bir olasılığa göre, bir görüntüdeki renkleri rasgele tersine çevirecektir.

In [ ]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (255-x)
  else:
    x
  return x

In [ ]:
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = random_invert(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0].numpy().astype("uint8"))
  plt.axis("off")

Ardından, [alt sınıflama(subclassing)](https://www.tensorflow.org/guide/keras/custom_layers_and_models) ile özel bir katman uygulayın:

In [ ]:
class RandomInvert(layers.Layer):
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor

  def call(self, x):
    return random_invert_img(x)

In [ ]:
_ = plt.imshow(RandomInvert()(image)[0])

Bu katmanların her ikisi de yukarıdaki seçenekler 1 ve 2'de açıklandığı gibi kullanılabilir.

## tf.image'i kullanma

Yukarıdaki Keras ön işleme yardımcı fonksiyonları uygundur. Ancak, daha hassas kontrol için `tf.data` ve `tf.image` kullanarak kendi veri arttırma işlem hatlarınızı veya katmanlarınızı yazabilirsiniz. 

Çiçekler (flowers) veri kümesi daha önce veri artırma ile yapılandırıldığından, yeni başlamak için yeniden içe aktaralım:

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Çalışmak için bir görüntü alın:

In [ ]:
image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

Orijinal ve artırılmış görüntüleri yan yana görselleştirmek ve karşılaştırmak için aşağıdaki fonksiyonu tanımlayalım:

In [ ]:
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

### Veri arttırma

#### Bir resmi çevirin

Bir görüntüyü `tf.image.flip_left_right` ile dikey veya yatay olarak çevirin:

In [ ]:
flipped = tf.image.flip_left_right(image)
visualize(image, flipped)

#### Gri tonlamalı bir görüntü

Bir görüntüyü `tf.image.rgb_to_grayscale` ile gri tonlama yapabilirsiniz:

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
_ = plt.colorbar()

#### Bir görüntüde saturation işlemi

Bir doygunluk (saturation) faktörü sağlayarak bir görüntünün "tf.image.adjust_saturation" ile saturation değeriyle oynayabilirsiniz:

In [ ]:
saturated = tf.image.adjust_saturation(image, 3)
visualize(image, saturated)

#### Görüntü parlaklığını (brightness) değiştirin

Bir parlaklık faktörü sağlayarak görüntünün parlaklığını `tf.image.adjust_brightness` ile değiştirin:

In [ ]:
bright = tf.image.adjust_brightness(image, 0.4)
visualize(image, bright)

#### Bir görüntüyü ortadan kırpma

`tf.image.central_crop` kullanarak görüntüyü merkezden istediğiniz görüntü parçasına kadar kırpın:

In [ ]:
cropped = tf.image.central_crop(image, central_fraction=0.5)
visualize(image, cropped)

#### Bir görüntüyü döndürme

`tf.image.rot90` ile bir resmi 90 derece döndürün:

In [ ]:
rotated = tf.image.rot90(image)
visualize(image, rotated)

### Rastgele Dönüşümler (Random transformations)

Uyarı: İki rastgele görüntü işlemi grubu vardır: `tf.image.random` ve `tf.image.stateless_random`. TF 1.x'ten eski RNG'leri kullandıkları için `tf.image.random*` işlemlerinin kullanılması kesinlikle önerilmez. 
Bunun yerine, lütfen bu eğitimde tanıtılan rastgele görüntü işlemlerini kullanın.

Görüntülere rastgele dönüşümler uygulamak, veri kümesini daha da genelleştirmeye ve genişletmeye yardımcı olabilir. Mevcut `tf.image` API, bu tür sekiz rastgele görüntü işlemi sağlar:

*   [`tf.image.stateless_random_brightness`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_brightness)
*   [`tf.image.stateless_random_contrast`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_contrast)
*   [`tf.image.stateless_random_crop`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_crop)
*   [`tf.image.stateless_random_flip_left_right`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_flip_left_right)
*   [`tf.image.stateless_random_flip_up_down`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_flip_up_down)
*   [`tf.image.stateless_random_hue`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_hue)
*   [`tf.image.stateless_random_jpeg_quality`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_jpeg_quality)
*   [`tf.image.stateless_random_saturation`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_saturation)

Bu rastgele görüntü işlemleri tamamen işlevseldir: çıktı yalnızca girdiye bağlıdır. Bu, yüksek performanslı, deterministik girdi boru hatlarında kullanımlarını kolaylaştırır. Her adımda bir `seed` değeri girilmesini gerektirirler. Aynı `seed` verildiğinde, kaç kez çağrıldıklarından bağımsız olarak aynı sonuçları verirler.

Not: `seed`, değerleri herhangi bir tam sayı olan `(2,)` boyutunda `Tensor`dür.

Aşağıdaki bölümlerde şunları yapacaksınız:

1.   Bir görüntüyü dönüştürmek için rastgele görüntü işlemlerini kullanma örneklerini gözden geçirin.
2.   Bir eğitim veri kümesine rastgele dönüşümlerin nasıl uygulanacağını gösterin.

#### Randomly change image brightness

Randomly change the brightness of `image` using `tf.image.stateless_random_brightness` by providing a brightness factor and `seed`. The brightness factor is chosen randomly in the range `[-max_delta, max_delta)` and is associated with the given `seed`.


#### Görüntü parlaklığını rastgele değiştirin

Bir parlaklık faktörü ve `seed` sağlayarak `tf.image.stateless_random_brightness` kullanarak `image`in parlaklığını rastgele değiştirin. Parlaklık faktörü `[-max_delta, max_delta)` aralığında rastgele seçilir ve verilen `seed` ile ilişkilendirilir.

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_brightness = tf.image.stateless_random_brightness(
      image, max_delta=0.95, seed=seed)
  visualize(image, stateless_random_brightness)

#### Randomly change image contrast

Randomly change the contrast of `image` using `tf.image.stateless_random_contrast` by providing a contrast range and `seed`. The contrast range is chosen randomly in the interval `[lower, upper]` and is associated with the given `seed`.


#### Rastgele görüntü kontrastını değiştirin

Bir kontrast aralığı ve `seed` sağlayarak `tf.image.stateless_random_contrast` kullanarak `image` kontrastını rastgele değiştirin. Kontrast aralığı `[lower, upper]` aralığında rastgele seçilir ve verilen `seed` ile ilişkilendirilir.

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_contrast = tf.image.stateless_random_contrast(
      image, lower=0.1, upper=0.9, seed=seed)
  visualize(image, stateless_random_contrast)

#### Bir resmi rastgele kırpın

Hedef `size` ve `seed` sağlayarak `tf.image.stateless_random_crop` kullanarak `image`ı rastgele kırpın. "Görüntü"den kırpılan kısım, rastgele seçilen bir konumdadır ve verilen `seed` ile ilişkilendirilir.

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_crop = tf.image.stateless_random_crop(
      image, size=[210, 300, 3], seed=seed)
  visualize(image, stateless_random_crop)

### Bir veri kümesine veri arttırma uygulama

Önceki bölümlerde değiştirilmiş olmaları ihtimaline karşı önce görüntü veri setini tekrar indirelim.

In [ ]:
(train_datasets, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Ardından, görüntüleri yeniden boyutlandırmak ve yeniden ölçeklendirmek için bir yardımcı fonksiyon tanımlayın. Bu fonksiyon, veri kümesindeki görüntülerin boyutunu ve ölçeğini birleştirmede kullanılacaktır:

In [ ]:
def resize_and_rescale(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = (image / 255.0)
  return image, label

Rastgele dönüşümleri görüntülere uygulayabilen `augment` fonksiyonunu da tanımlayalım. Bu fonksiyon bir sonraki adımda veri setinde kullanılacaktır.

In [ ]:
def augment(image_label, seed):
  image, label = image_label
  image, label = resize_and_rescale(image, label)

  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)

  # Make a new seed.
  new_seed = tf.random.experimental.stateless_split(seed, num=1)[0, :]

  # Random crop back to the original size.
  image = tf.image.stateless_random_crop(
      image, size=[IMG_SIZE, IMG_SIZE, 3], seed=seed)
  
  # Random brightness.
  image = tf.image.stateless_random_brightness(
      image, max_delta=0.5, seed=new_seed)
  image = tf.clip_by_value(image, 0, 1)
  
  return image, label

#### Seçenek 1: tf.data.experimental.Counter'ı Kullanma

Bir `tf.data.experimental.Counter` nesnesi oluşturun (buna `counter` diyelim) ve veri kümesini `(counter, counter)` ile `Dataset.zip` oluşturun. Bu, veri kümesindeki her görüntünün, daha sonra rastgele dönüşümler için `seed` değeri olarak `arttırma(augment)` işlevine aktarılabilen `counter`a dayalı benzersiz bir değerle (`(2,)` şeklinde) ilişkilendirilmesini sağlayacaktır. .

In [ ]:
# Create a `Counter` object and `Dataset.zip` it together with the training set.
counter = tf.data.experimental.Counter()
train_ds = tf.data.Dataset.zip((train_datasets, (counter, counter)))

`augment` işlevini eğitim veri kümesiyle eşleştirin:

In [ ]:
train_ds = (
    train_ds
    .shuffle(1000)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
val_ds = (
    val_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
test_ds = (
    test_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

#### Seçenek 2: tf.random.Generator'ı Kullanma

- Başlangıç değeri `seed` olan bir `tf.random.Generator` nesnesi oluşturun. Aynı oluşturucu nesne üzerinde `make_seeds` işlevinin çağrılması her zaman yeni, benzersiz bir `seed` değeri döndürür.

- Aşağıdaki bir fonlsiyon tanımlayın: 1) `make_seeds` fonksiyonunu çağırsın; ve 2) rastgele dönüşümler için yeni oluşturulan `seed` değerini `augment` işlevine iletir.

Not: `tf.random.Generator` nesneleri, RNG durumunu bir `tf.Variable`'da saklar; bu, bir kontrol noktası (checkpoint) olarak veya bir SavedModel'de kaydedilebileceği anlamına gelir.

In [ ]:
# Create a generator.
rng = tf.random.Generator.from_seed(123, alg='philox')

In [ ]:
# Create a wrapper function for updating seeds.
def f(x, y):
  seed = rng.make_seeds(2)[0]
  image, label = augment((x, y), seed)
  return image, label

`f` fonksiyonu eğitim veri kümesiyle ve `resize_and_rescale` işlevini doğrulama ve test kümeleriyle eşleyin:

In [ ]:
train_ds = (
    train_datasets
    .shuffle(1000)
    .map(f, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
val_ds = (
    val_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
test_ds = (
    test_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

Bu veri kümeleri şimdi daha önce gösterildiği gibi bir modeli eğitmek için kullanılabilir.